In [1]:
# some imports
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))

# Python ≥3.5 is required
import sys

assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn

assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf

assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
# % matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
plt.rc('font', size=12)
plt.rc('figure', figsize=(12, 5))

# Settings for the visualizations
#import seaborn as sns
#sns.set_style("whitegrid")
#sns.set_context("notebook", font_scale=1, rc={"lines.linewidth": 2,'font.family': [u'times']})

import pandas as pd

pd.set_option('display.max_rows', 25)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 50)

# Ignore useless warnings (see SciPy issue #5998)
import warnings

warnings.filterwarnings(action="ignore", message="^internal gelsd")
# Others
import cv2
import tqdm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.model_selection import train_test_split
from skimage.io import imread

 ## Import data

In [2]:
# Paths
X_train_path = './dataset/train/train/train/'
X_test_path = './dataset/test/test/test/'
dataset_csv = './dataset/train.txt'
dataset_train = './dataset/train/train'
dataset_test = './dataset/test/test'


In [3]:
_STOP = 20000


def preprocess(img_paths, dataset_path, preprocess_img_method=None):
    print("loading data")
    data = []
    stop = 0
    for img_path in tqdm.tqdm(img_paths):
        path = os.path.realpath(os.path.join(dataset_path, img_path[1:]))
        img = imread(path)
        if preprocess_img_method:
            img = preprocess_img_method(img)
        else:
            # img = cv2.resize(img,(224,224),cv2.INTER_AREA)
            img = img / 255.  #normalize
        data.append(img)
        stop += 1
        if (stop == _STOP):
            break
    print("loading data done")
    return data


def preprocess_img_vgg(img):
    return preprocess_input(img)


In [4]:
df = pd.read_csv(dataset_csv, delimiter='\ ', header=None)
y_train_full = np.array(df[1])[:_STOP]
img_paths = df[0]
X_train_full = preprocess(df[0], dataset_train, preprocess_img_method=None)
X_train_full = np.stack(X_train_full)

# Split dataset
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42, test_size=0.1)
X_train = X_train.astype("float32")
X_valid = X_valid.astype("float32")
y_train = y_train.astype("float32")
y_valid = y_valid.astype("float32")

C:\ProgramData\Anaconda3\envs\tfgpu\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


loading data


100%|██████████| 15000/15000 [00:10<00:00, 1381.49it/s]


loading data done


In [5]:
no_classes = np.max(y_train_full) + 1
print(no_classes)

30


In [6]:
def root_mean_squared_error(y_true, y_pred):
    y1 = tf.cast(tf.math.argmax(y_pred, axis=-1), tf.float32)
    return tf.sqrt(tf.cast(tf.math.reduce_mean(tf.square(y1 - y_true)), tf.float64))

def metric_root_mean_squared_error(y_true, y_pred):
    y1 = tf.math.argmax(y_true, axis=-1)
    y2 = tf.math.argmax(y_pred, axis=-1)
    return tf.sqrt(tf.cast(tf.math.reduce_mean(tf.square(y1 - y2)), tf.float64))

In [7]:
def get_uncompiled_model():
    model = tf.keras.models.Sequential()
    # model.add(tf.keras.layers.Flatten(input_shape=[158, 158]))
    # model.add(tf.keras.layers.Dense(512, activation="relu", kernel_initializer="he_normal"))
    # model.add(tf.keras.layers.Dense(512, activation="relu", kernel_initializer="he_normal"))
    # model.add(tf.keras.layers.Dense(256, activation="relu", kernel_initializer="he_normal"))
    # model.add(tf.keras.layers.Dense(256, activation="relu", kernel_initializer="he_normal"))
    # model.add(tf.keras.layers.Dense(64, activation="relu", kernel_initializer="he_normal"))
    # model.add(tf.keras.layers.Dropout(0.4))
    # model.add(tf.keras.layers.Dense(no_classes, activation="softmax"))
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=[158, 158]))
    model.add(tf.keras.layers.Dense(250, activation="relu", kernel_initializer="he_normal"))
    model.add(tf.keras.layers.Dense(150, activation="relu", kernel_initializer="he_normal"))
    model.add(tf.keras.layers.Dense(50, activation="relu", kernel_initializer="he_normal"))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(no_classes + 1, activation="softmax"))
    return model


def get_compiled_model():
    model = get_uncompiled_model()
    model.compile(
        # optimizer=keras.optimizers.SGD(lr=1e-5, decay=(5 * 1e-5), momentum=0.95),  # Optimizer
        optimizer=keras.optimizers.SGD(),  # Optimizer
        # Loss function to minimize
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        # List of metrics to monitor
        metrics=["accuracy", root_mean_squared_error]
    )
    return model

In [53]:
# Prepare a directory to store all the checkpoints.
checkpoint_dir = "./ckpt/mcc-2-simple"
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)


def make_or_restore_model():
    # Either restore the latest model, or create a fresh one
    # if there is no checkpoint available.
    checkpoints = [checkpoint_dir + "/" + name for name in os.listdir(checkpoint_dir)]

    if checkpoints:
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        print("Restoring from", latest_checkpoint)
        return keras.models.load_model(latest_checkpoint)
    print("Creating a new model")
    return get_compiled_model()


def restore_model():
    checkpoints = [checkpoint_dir + "/" + name for name in os.listdir(checkpoint_dir)]
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        print("Restoring from", latest_checkpoint)
        return keras.models.load_model(latest_checkpoint)


def restore_model_weights_only():
    model = get_compiled_model()
    # checkpoints = [checkpoint_dir + "/" + name for name in os.listdir(checkpoint_dir)]
    latest = tf.train.latest_checkpoint(checkpoint_dir)
    print("Restoring from", latest)
    model.load_weights(latest)
    return model


def restore_model_weights_only_uncompiled():
    model = get_uncompiled_model()
    # checkpoints = [checkpoint_dir + "/" + name for name in os.listdir(checkpoint_dir)]
    latest = tf.train.latest_checkpoint(checkpoint_dir)
    print("Restoring from", latest)
    model.load_weights(latest)
    return model


### Early Stopping
from tensorflow.keras.callbacks import EarlyStopping

# configure early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=25)
###

### init callbacks
callbacks = [
    keras.callbacks.ModelCheckpoint(
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        # The saved model name will include the current epoch.
        filepath=checkpoint_dir + "/ckpt_{epoch}_loss={loss:.4f}_val_loss={val_loss:.4f}",
        save_best_only=True,  # Only save a model if `val_loss` has improved.
        monitor="val_loss",
        save_weights_only=True,
        mode="min",
        verbose=1,
    ),
    early_stopping
]

In [68]:
model = get_compiled_model()
model.summary()

Model: "sequential_49"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_24 (Flatten)         (None, 24964)             0         
_________________________________________________________________
dense_96 (Dense)             (None, 250)               6241250   
_________________________________________________________________
dense_97 (Dense)             (None, 150)               37650     
_________________________________________________________________
dense_98 (Dense)             (None, 50)                7550      
_________________________________________________________________
dropout_24 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_99 (Dense)             (None, 31)                1581      
Total params: 6,288,031
Trainable params: 6,288,031
Non-trainable params: 0
___________________________________________

In [69]:
model = restore_model_weights_only_uncompiled()
model.compile(
    # optimizer=keras.optimizers.SGD(lr=1e-5, decay=(5 * 1e-5), momentum=0.95),  # Optimizer
    optimizer=keras.optimizers.SGD(lr=5e-4, decay=(5 * 1e-5), momentum=0.95),  # Optimizer
    # Loss function to minimize
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    # List of metrics to monitor
    metrics=["accuracy", root_mean_squared_error]
)

Restoring from ./ckpt/mcc-2-simple\ckpt_20_loss=1.9509_val_loss=1.8287


In [70]:
print("Fit model on training data")
history = model.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=50,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    # validation_data=(X_valid, y_valid),
    validation_split=0.2,
    shuffle=True,
    callbacks=callbacks
)
pd.DataFrame(history.history).plot(figsize=(8, 5))

Fit model on training data
Epoch 1/50
325/338 [===========================>..] - ETA: 0s - loss: 1.9616 - accuracy: 0.2553 - root_mean_squared_error: 12.2196

In [13]:
predict = model.predict(X_train[:1])
predict1 = model.predict(X_train[2][np.newaxis, :, :])
print(predict)

[[2.7371758e-08 1.2714499e-07 7.9844830e-07 2.5785451e-07 5.6970583e-07
  3.0781959e-07 2.9688763e-08 4.7324932e-08 1.2521253e-06 1.8415471e-05
  1.2448820e-04 1.6954817e-03 1.0855487e-02 9.3436971e-02 1.1569290e-01
  3.0096871e-01 2.7902430e-01 9.0469725e-02 7.4870646e-02 3.0672496e-02
  1.7469306e-03 3.2303610e-04 8.5192405e-05 3.2263042e-06 1.4271055e-06
  2.8108789e-06 1.2762247e-06 1.7258907e-06 5.9130468e-07 3.2936089e-07
  3.6685654e-07]]


In [71]:
preds = []
n = len(y_valid)
for data in X_valid[:n]:
    pred = model.predict(data[np.newaxis, :, :])
    preds.append((np.argmax(pred)))

##
vals = y_valid[:n]
rmse = [np.sqrt(np.sum(np.square(preds - vals)) / n)]
print(rmse)

[2.0250102880397094]


In [15]:
print(rmse)

[2.138535324312725]


# Tests


In [16]:
# Save Model

In [25]:
model.save('./ckpt/aa2/', save_format='tf')

INFO:tensorflow:Assets written to: ./ckpt/aa2/assets


In [26]:
model = tf.keras.models.load_model('./ckpt/aa2/')

ValueError: Unable to restore custom object of type _tf_keras_metric currently. Please make sure that the layer implements `get_config`and `from_config` when saving. In addition, please use the `custom_objects` arg when calling `load_model()`.

In [21]:
## TEST restore weights
model = restore_model_weights_only()
# Re-evaluate the model
loss, acc, rmse = model.evaluate(X_valid, y_valid, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))
print(rmse)


Restoring from ./ckpt\ckpt_47_loss=1.9761_val_loss=1.8614
47/47 - 0s - loss: 1.8698 - accuracy: 0.2947 - root_mean_squared_error: 12.2345
Restored model, accuracy: 29.47%
12.234536170959473
